### Total OD and OD for three types of trips

In [1]:
import numpy as np
import pandas as pd
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import os
import datetime
from tqdm import tqdm
import pickle
import copy

In [2]:
inFile = '../Metro/metroData_ts_v2.csv'
outFile = '../Metro/metroData_ts_v3.csv' 
#os.system('sort --header -t, -k1,1n -k3,3n -k4,4n ' + inFile + ' > ' + outFile) # sorted by user and by time
os.system('sort -t, -k2,2n -k3,3n -k4,4n ' + inFile + ' > ' + outFile) # Start time is added as a new col

0

In [2]:
def get_tripLabel(UserID, inStation, outStation, station_lightLabels):
    HomeStationID = station_lightLabels[UserID]['h']
    WorkStationID = station_lightLabels[UserID]['w']

    TripLabel = 3
    if (inStation, outStation) == (HomeStationID, WorkStationID) or (inStation, outStation) == (WorkStationID, HomeStationID) :
        TripLabel  = 0
    elif inStation == HomeStationID or outStation == HomeStationID :
        TripLabel = 1
    elif inStation == WorkStationID or outStation == WorkStationID:
        TripLabel = 2
        
    return TripLabel


In [3]:
def travelODFlowGrouped(interval): 

    # metro schedule  6:00-23:00 
    startHour = 6
    endHour = 23
    timeslotsPerDay = (endHour - startHour)*60/interval  

    #################
    # Create Day Bins
    dayIndex = {} # {yearmonthday: index}
    count = 0
    for month in range(5, 9):
        yearmonth = '2017' + str(month).zfill(2)
        for day in range(32):
            dd = yearmonth + str(day).zfill(2)
            try:
                daytime = datetime.datetime.strptime(dd, '%Y%m%d')
                dayIndex[dd] = count
                count += 1
            except:
                continue

    ##################
    # Load Station Info
    # stationsData = open("results/metroStations.csv", 'r') # should be new ID
    stationsData = open ("../Metro/station_transInfo_cleaned.csv", 'r') 
    next(stationsData)
    stations = [] # list of station id
    for row in stationsData:
        row = row.rstrip().split(',')
        station = int(row[1])
        stations.append(station)

    station_dict = {value: index for index, value in enumerate(stations)}
    station_num=len(stations)
    zeroOD=np.zeros((station_num,station_num))

    # transferStation, transferStationToNewID = pickle.load(open("results/transferStations.pkl", 'rb'))
    transferStation, transferStationToNewID = pickle.load(open("../Metro/transferStations.pkl",'rb'))

    # sort stations
    stations.sort()

    ##################
    # Load User Info
    with open('/data6/peiyan/SH-METR/results/station_lightLabels.pkl','rb') as infile:
        station_lightLabels = pickle.load(infile)

    
    ##################
    # Extract Demand
    # load data
    #inFile = "/media/xu/OS/Data/SmartCardData/metroData_ts.csv"
    inFile = "../Metro/metroData_ts_v3.csv"
    inData = open(inFile, 'r')
    next(inData)
    inDataList = list(inData)
    OD_mat = copy.deepcopy(zeroOD) 
    COD_mat = copy.deepcopy(zeroOD)
    HO_OD_mat = copy.deepcopy(zeroOD)
    WO_OD_mat = copy.deepcopy(zeroOD)
    currentTimeslot = 0
    currentDay = ''
    slot_startTime = datetime.datetime.strptime('06:00:00', "%H:%M:%S") 
    slot_endTime = slot_startTime + datetime.timedelta (minutes = interval)
    slot_startTime = slot_startTime.strftime("%H%M%S")
    slot_endTime = slot_endTime.strftime("%H%M%S")      

    outFile = "/data6/peiyan/SH-METR/results/metroData_ODFlow_"+str(interval)+".csv"
    outData = open(outFile, 'w')
    outData.writelines("date, timeslot, startTime, endTime, originStation, destinationStation, Flow, CFlow, HOFlow, WOFlow\n")  

    for row in tqdm(inDataList):
        row = row.rstrip().split(',')
        userID = int(row[0])
        transDay = row[1]
        InTime = row [2]
        outTime = row [3]
        transHour = int(InTime[:2])
        if transHour < startHour or transHour >= endHour:     # metro schedule  6:00-23:00 
            continue
        minute = int(InTime[2:4])
        try:
            dayIdx = dayIndex[transDay]   
        except:
            continue

        timeslot = int(dayIdx * timeslotsPerDay + (transHour - startHour) * 60 / interval + minute // interval)

        if timeslot < 0:
            continue

        if InTime > outTime: # For save
            continue

        if currentDay == '':
            currentDay = transDay

        if timeslot != currentTimeslot:
            # save OD mat of last timeslot
            for row_id in range(station_num):
                for col_id in range(station_num):
                    if row_id == col_id:
                        continue
                    OD_Flow = str(int(OD_mat[row_id,col_id]))
                    COD_Flow = str(int(COD_mat[row_id,col_id]))
                    HO_OD_Flow = str(int(HO_OD_mat[row_id,col_id]))
                    WO_OD_Flow = str(int(WO_OD_mat[row_id,col_id]))
                    if int(OD_Flow) != 0:
                        outData.writelines(','.join([currentDay, str(currentTimeslot), slot_startTime ,slot_endTime, str(stations[row_id]), str(stations[col_id]), OD_Flow, COD_Flow, HO_OD_Flow, WO_OD_Flow]) + '\n')

            # initilize
            OD_mat = copy.deepcopy(zeroOD)
            COD_mat = copy.deepcopy(zeroOD)
            HO_OD_mat = copy.deepcopy(zeroOD)
            WO_OD_mat= copy.deepcopy(zeroOD)

            currentTimeslot = timeslot
            currentDay = transDay
            slot_startTime = datetime.datetime.strptime('06:00:00', "%H:%M:%S") + datetime.timedelta(minutes = interval * (timeslot % timeslotsPerDay))
            slot_endTime = slot_startTime + datetime.timedelta (minutes = interval)
            slot_startTime = slot_startTime.strftime("%H%M%S")
            slot_endTime = slot_endTime.strftime("%H%M%S")

        inStation = int(row[4])
        outStation = int(row[5])

        # # Updata Station ID
        # try:
        #     inStation_new = transferStationToNewID[inStation]
        # except:
        #     inStation_new = inStation
        # try:
        #     outStation_new = transferStationToNewID[outStation]
        # except:
        #     outStation_new = outStation

        # if inStation_new not in set(stations) or outStation_new not in set(stations):
        #     continue

        OD_mat[station_dict[inStation], station_dict[outStation]]+=1
        
        try:
            tripLabel = get_tripLabel(userID, inStation, outStation, station_lightLabels)
            if tripLabel == 0:
                COD_mat[station_dict[inStation], station_dict[outStation]]+=1
            elif tripLabel == 1:
                HO_OD_mat[station_dict[inStation], station_dict[outStation]]+=1
            elif tripLabel == 2 :
                WO_OD_mat[station_dict[inStation], station_dict[outStation]]+=1
        except:
             pass # key missing   


    # last timeslot
    # save the od matrix of the last timeslot
    for row_id in range(station_num):
        for col_id in range(station_num):
            if row_id == col_id:
                continue
            OD_Flow = str(int(OD_mat[row_id,col_id]))
            COD_Flow = str(int(COD_mat[row_id,col_id]))
            HO_OD_Flow = str(int(HO_OD_mat[row_id,col_id]))
            WO_OD_Flow = str(int(WO_OD_mat[row_id,col_id]))
            if  int(OD_Flow) != 0:
                outData.writelines(','.join([currentDay, str(currentTimeslot), slot_startTime ,slot_endTime, str(stations[row_id]), str(stations[col_id]), OD_Flow, COD_Flow, HO_OD_Flow, WO_OD_Flow]) + '\n')

    inData.close()
    outData.close()

In [4]:
travelODFlowGrouped(interval=10)

100%|██████████| 736847403/736847403 [1:38:06<00:00, 125175.66it/s] 
